<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/3d1d_advdiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2025-09-23 16:40:07--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-09-23 16:40:08 (73.0 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9732719d/releases/pybi

In [ ]:
# @title Convergence test: 3D-dG,1D-dG time-dependent advection-diffusion

from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Testing

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

T    = 1.0

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Omega_boundaries = MeshFunction("size_t", mesh_3d, mesh_3d.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Omega_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Omega_boundaries, 2)
  Lambda = EmbeddedMesh(Omega,1)
  Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Lambda_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Lambda_boundaries, 2)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "DG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)


  # === Initial concentration ===
  cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dsOmega = Measure("ds", domain=mesh_3d, subdomain_data = Omega_boundaries)
  dSOmega = Measure("dS", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  dSLambda = Measure("dS", domain=Lambda)
  n3d      = FacetNormal(mesh_3d)
  n1d      = FacetNormal(Lambda)
  h3d        = CellDiameter(mesh_3d)
  h1d        = CellDiameter(Lambda)
  alpha3d    = Constant(50.0)
  alpha1d    = Constant(50.0)
  alpha_boundary = Constant(50.0)
  epsilon  = Constant(1.0)
  perf1    = 1.0
  perf3    = 1.0

  # === Velocity terms ===
  q1 = Constant(1.0)
  q3 = as_vector((0,0,q1))
  q3_pos = dot(q3,n3d('+'))

  # === Upwind ===
  u1_up = conditional(ge(q1, 0.0), u1('-'), u1('+'))
  u3_up = conditional(ge(q3_pos, 0.0), u3('-'), u3('+'))

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    cinlet.t = t

    # === Exact solutions ===
    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)
    u1_inlet_expr    = Expression('(sin(pi*x[2])+2)*t', degree=7, t=t)  # right now same as u1_exact but can be changed in future for diff circumstance
    du1ds_exact_expr = Expression('pi*t*cos(pi*x[2])', degree=7, t=t)

    theta0 = Expression('area*(du - q*(u - uin))',
                    degree=7, area=D_area, q=q1,
                    du=du1ds_exact_expr, u=u1_exact_expr, uin=u1_inlet_expr)

    thetaN = Expression('area*du', degree=7, area=D_area, du=du1ds_exact_expr)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? '
                    '((kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) '             # time derivative
                    '+ (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*t*sin(pi*x[2]) '    # diffusion
                    '+ (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pi*t*q*cos(pi*x[2]) )'        # advection
                    ': ((kappa/(1+kappa))*pow(pi,2)*t*sin(pi*x[2]) '            # diffusion
                    '+ (kappa/(1+kappa))*(sin(pi*x[2])+2) '                     # time derivative
                    '+ (kappa/(1+kappa))*pi*t*q*cos(pi*x[2]) ) )',                  # advection
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t,
                    q = q1)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)'  # coupling
        ' + area * pi * t * q * cos(pi*x[2])',                # advection
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t,
        q=q1
    )

    # === Variational Forms ===
    a00 = inv_dt * (inner(u3, v3) * dxOmega) \
          + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda \
          - epsilon * dot(avg(grad(v3)),jump(u3,n3d))* dSOmega \
          - dot(jump(v3,n3d),avg(grad(u3))) * dSOmega \
          + (alpha3d/avg(h3d)) * dot(jump(v3,n3d),jump(u3,n3d)) * dSOmega \
          - epsilon * dot(grad(v3),u3*n3d) * dsOmega - dot(v3*n3d,grad(u3)) * dsOmega \
          + (alpha_boundary/h3d)*v3*u3*dsOmega \
          - inner(q3, grad(v3)) * u3 * dxOmega \
          + q3_pos * u3_up * jump(v3) * dSOmega \
          + dot(q3,n3d) * u3 * v3 * dsOmega(2)
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
          + D_area * inner(grad(u1), grad(v1)) * dxLambda \
          - D_area * inner(avg(grad(u1)), jump(v1, n1d)) * dSLambda \
          - epsilon * D_area * inner(avg(grad(v1)), jump(u1, n1d)) * dSLambda \
          + (alpha1d/avg(h1d)) * jump(u1) * jump(v1) * dSLambda \
          + kappa * D_perimeter * u1 * v1 * dxLambda \
          - D_area * q1 * u1 * v1.dx(2) * dxLambda \
          + D_area*q1*u1_up*jump(v1)*dSLambda \
          + D_area * q1 * u1 * v1 * dsLambda(2)


    L0 = inner(f3, v3) * dxOmega + inv_dt * (inner(uh3d_prev, v3) * dxOmega) \
        - epsilon * dot(grad(v3),u3_exact*n3d) * dsOmega + (alpha_boundary/h3d)*v3*u3_exact*dsOmega \
        + dot(q3,n3d) * u3_exact * v3 * dsOmega(1)
    L1 = inner(f1, v1) * dxLambda \
        + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + D_area * q1 * cinlet * v1 * dsLambda(1)  \
        - theta0*v1*dsLambda(1) + thetaN*v1*dsLambda(2)


    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

   # bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [[], []]  # Boundaries now included in variational form for dG

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)


/usr/local/lib/python3.12/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing
Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_426052d30967b565ba5828eeea7a01043e99f3c8

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00146103 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.202428 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_99e64e64a0b0bf7efa2745c3166b98a12c5ca04c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00138998 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0236313 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:---------------------------

...

  0 KSP unpreconditioned resid norm 2.738826329335e+00 true resid norm 2.738826329335e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.519421494502e-01 true resid norm 1.519421494502e-01 ||r(i)||/||b|| 5.547710266355e-02
  2 KSP unpreconditioned resid norm 3.165683000993e-02 true resid norm 3.165683000993e-02 ||r(i)||/||b|| 1.155853865974e-02
  3 KSP unpreconditioned resid norm 1.782787880268e-02 true resid norm 1.782787880268e-02 ||r(i)||/||b|| 6.509313355042e-03
  4 KSP unpreconditioned resid norm 7.445549532602e-03 true resid norm 7.445549532602e-03 ||r(i)||/||b|| 2.718518313065e-03
  5 KSP unpreconditioned resid norm 2.392357679205e-03 true resid norm 2.392357679205e-03 ||r(i)||/||b|| 8.734974005400e-04
  6 KSP unpreconditioned resid norm 1.454001556870e-03 true resid norm 1.454001556870e-03 ||r(i)||/||b|| 5.308849054415e-04
  7 KSP unpreconditioned resid norm 6.099206831601e-04 true resid norm 6.099206831598e-04 ||r(i)||/||b|| 2.226941798489e-04
  8 KSP 

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 472.28it/s]


  0 KSP unpreconditioned resid norm 2.709280241127e+00 true resid norm 2.709280241127e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.487717480864e-01 true resid norm 1.487717480864e-01 ||r(i)||/||b|| 5.491190827290e-02
  2 KSP unpreconditioned resid norm 3.049611402337e-02 true resid norm 3.049611402337e-02 ||r(i)||/||b|| 1.125616817354e-02
  3 KSP unpreconditioned resid norm 1.686232895052e-02 true resid norm 1.686232895052e-02 ||r(i)||/||b|| 6.223914637750e-03
  4 KSP unpreconditioned resid norm 6.659561627772e-03 true resid norm 6.659561627772e-03 ||r(i)||/||b|| 2.458055658724e-03
  5 KSP unpreconditioned resid norm 2.242973543196e-03 true resid norm 2.242973543196e-03 ||r(i)||/||b|| 8.278853952231e-04
  6 KSP unpreconditioned resid norm 1.406586389643e-03 true resid norm 1.406586389643e-03 ||r(i)||/||b|| 5.191734573231e-04
  7 KSP unpreconditioned resid norm 6.296557347720e-04 true resid norm 6.296557347719e-04 ||r(i)||/||b|| 2.324070154182e-04
  8 KSP 

Averaging over 64 cells: 100%|██████████| 64/64 [00:00<00:00, 470.15it/s]


  0 KSP unpreconditioned resid norm 2.694428787402e+00 true resid norm 2.694428787402e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.474750220688e-01 true resid norm 1.474750220688e-01 ||r(i)||/||b|| 5.473331592892e-02
  2 KSP unpreconditioned resid norm 2.983679224864e-02 true resid norm 2.983679224864e-02 ||r(i)||/||b|| 1.107351301624e-02
  3 KSP unpreconditioned resid norm 1.624356208584e-02 true resid norm 1.624356208584e-02 ||r(i)||/||b|| 6.028573537289e-03
  4 KSP unpreconditioned resid norm 6.143740661944e-03 true resid norm 6.143740661943e-03 ||r(i)||/||b|| 2.280164423224e-03
  5 KSP unpreconditioned resid norm 2.196385732576e-03 true resid norm 2.196385732576e-03 ||r(i)||/||b|| 8.151582045312e-04
  6 KSP unpreconditioned resid norm 1.406292137231e-03 true resid norm 1.406292137233e-03 ||r(i)||/||b|| 5.219258878943e-04
  7 KSP unpreconditioned resid norm 6.663504898348e-04 true resid norm 6.663504898348e-04 ||r(i)||/||b|| 2.473067734988e-04
  8 KSP 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp conv_study1d.txt "/content/drive/My Drive/conv_study1d.txt"
!cp conv_study3d.txt "/content/drive/My Drive/conv_study3d.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
